<a href="https://colab.research.google.com/github/tzf101/190041101-CSE-4302/blob/master/utils_notebook/BDA_framework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installation

### EDA

In [1]:
!pip install bnlp

  Preparing metadata (setup.py) ... done
  Created wheel for bnlp: filename=bnlp-0.8-py3-none-any.whl size=32803 sha256=b8faeb88a7695126751899b47af994055ba88fe46621080786ad4625d0f538cc
  Stored in directory: /root/.cache/pip/wheels/64/c7/7e/272284eb8c800fbb5306bdf7904ae9f99a64812fc615f471b2
Successfully built bnlp


In [2]:
!pip install bnlp-toolkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.7 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171033 sha256=c8e74016b624d25be3576ee04810b3031da2496438807e942d39f321fb456788
  Stored in directory: /root/.cache/pip/wheels/31/8a/8c/315c9e5d7773f74b33d5ed33f075b49c6eaeb7cedbb86e2cf8
Successfully built emoji


### Transformer

In [3]:
!pip install git+https://github.com/csebuetnlp/normalizer

  Cloning https://github.com/csebuetnlp/normalizer to /tmp/pip-req-build-j9k_8e33
  Running command git clone --filter=blob:none --quiet https://github.com/csebuetnlp/normalizer /tmp/pip-req-build-j9k_8e33
  Resolved https://github.com/csebuetnlp/normalizer to commit d405944dde5ceeacb7c2fd3245ae2a9dea5f35c9
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for normalizer: filename=normalizer-0.0.1-py3-none-any.whl size=6859 sha256=cc87cc50df8185ad3355713aa7a9c44b7f793be9ffffc52d80075e7500c97426
  Stored in directory: /tmp/pip-ephem-wheel-cache-607oybnk/wheels/2e/79/9c/cd96d490298305d51d2da11484bb2c25fd1f759a6906708282
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186460 sha256=dc53799d6c3396cc273ed182306403cc0adac30

In [4]:
!pip install sentencepiece

# Imports

### EDA

In [5]:
import random
import re
from bnlp import BengaliCorpus as corpus
from bnlp import BengaliWord2Vec

punkt not found. downloading...


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


### Transformers

In [6]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from normalizer import normalize
import sentencepiece as spm
import torch
from transformers import BertTokenizer  # From Hugging Face's transformers library
from transformers import AutoTokenizer, AutoModelForMaskedLM

# Text Cleaning

In [7]:
class CleanText:
    def __init__(self):
        self.punctuations = set(corpus.punctuations + "‘’")
        self.stopwords = set(corpus.stopwords)

    def remove_digits(self, text):
        return re.sub(r'[০-৯]+\d+', '', text).strip()

    def remove_punctuations(self, text, replace_with=" "):
        for punct in self.punctuations:
            text = text.replace(punct, replace_with)
        return ' '.join(text.split())

    def remove_stopwords(self, text):
        words = text.split()
        return ' '.join(word for word in words if word.lower() not in self.stopwords)

# Get Synonyms

In [8]:
def get_synonyms(word):
    synonyms = set()
    bwv = BengaliWord2Vec()
    similar_words = bwv.get_most_similar_words(word, topn=10)
    for word in similar_words:
        synonyms.add(word[0])
    if word in synonyms:
        synonyms.remove(word)
    return list(synonyms)

# Base Class

In [9]:
class BaseTextAugmentor:
    def __init__(self, clean_text):
        self.clean_text = clean_text

    def augment(self, text):
        raise NotImplementedError("This method should be implemented by subclasses")

# Paraphrase

In [10]:
class BanglaParaphraseAugmentor(BaseTextAugmentor):
    def __init__(self, clean_text, model_name="csebuetnlp/banglat5_banglaparaphrase"):
        super().__init__(clean_text)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

    def augment(self, text, debug=False):
        input_tokens = self.tokenizer(normalize(text), return_tensors="pt").input_ids
        generated_tokens = self.model.generate(input_tokens)
        decoded_tokens = self.tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
        if debug:
            decoded_tokens += '(pp)'
        return decoded_tokens

# BT

In [11]:
class BTAugmentor(BaseTextAugmentor):
    def __init__(self, clean_text, modelb2e_name="csebuetnlp/banglat5_nmt_bn_en", modele2b_name="csebuetnlp/banglat5_nmt_en_bn"):
        super().__init__(clean_text)
        self.modelb2e = AutoModelForSeq2SeqLM.from_pretrained(modelb2e_name)
        self.tokenizerb2e = AutoTokenizer.from_pretrained(modelb2e_name, use_fast=False)

        self.modele2b = AutoModelForSeq2SeqLM.from_pretrained(modele2b_name)
        self.tokenizere2b = AutoTokenizer.from_pretrained(modele2b_name, use_fast=False)

    def augment(self, text, debug=False):
        input_ids = self.tokenizerb2e(normalize(text), return_tensors="pt").input_ids
        generated_tokens = self.modelb2e.generate(input_ids)
        decoded_tokens = self.tokenizerb2e.batch_decode(generated_tokens, skip_special_tokens=True)[0]

        input_ids = self.tokenizere2b(normalize(decoded_tokens), return_tensors="pt").input_ids
        generated_tokens = self.modele2b.generate(input_ids)
        decoded_tokens = self.tokenizere2b.batch_decode(generated_tokens, skip_special_tokens=True)[0]

        if debug:
            decoded_tokens += '(bt)'
        return decoded_tokens

# Synonym Replacement

In [12]:
class SynonymReplacementAugmentor(BaseTextAugmentor):
    def __init__(self, clean_text, stopwords):
        super().__init__(clean_text)
        self.stopwords = stopwords

    def augment(self, text, n, debug=False):
        words = text.split()
        new_words = words.copy()
        random_word_list = list(set([word for word in words if word not in self.stopwords]))
        random.shuffle(random_word_list)
        num_replaced = 0
        for random_word in random_word_list:
            synonyms = get_synonyms(random_word)
            if len(synonyms) >= 1:
                synonym = random.choice(list(synonyms))
                new_words = [synonym if word == random_word else word for word in new_words]
                num_replaced += 1
            if num_replaced >= n:
                break

        output = ' '.join(new_words)
        if debug:
            output += "(sr)"
        return output

# MLM

In [13]:
class MLMAugmentor(BaseTextAugmentor):
    def __init__(self, clean_text, model_name="csebuetnlp/banglabert_generator"):
        super().__init__(clean_text)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForMaskedLM.from_pretrained(model_name)

    def augment(self, text, num_mask=3, debug=False):
        token_ids = self.tokenizer.encode(text, return_tensors='pt')[0]
        indices_to_mask = random.sample(range(1, len(token_ids) - 1), num_mask)
        masked_token_ids = [self.tokenizer.mask_token_id if i in indices_to_mask else token_id for i, token_id in enumerate(token_ids)]
        masked_text = self.tokenizer.decode(masked_token_ids)

        input_ids = self.tokenizer(masked_text, return_tensors='pt').input_ids
        predictions = self.model(input_ids).logits
        masked_indices = (input_ids == self.tokenizer.mask_token_id).nonzero(as_tuple=True)[1]

        for idx in masked_indices:
            predicted_token_id = predictions[0, idx].topk(1).indices.item()
            input_ids[0, idx] = predicted_token_id

        unmasked_text = self.tokenizer.decode(input_ids[0], skip_special_tokens=True)
        if debug:
            unmasked_text += "(mlm)"
        return unmasked_text

# Random Deletion

In [14]:
# Probability based
class RandomDeletionAugmentor(BaseTextAugmentor):
    def augment(self, text, p=0.3, debug=False):
        words = text.split()
        if len(words) == 1:
            return words

        new_words = []
        for word in words:
            r = random.uniform(0, 1)
            if r > p:
                new_words.append(word)

        if len(new_words) == 0:
            rand_int = random.randint(0, len(words) - 1)
            return [words[rand_int]]

        output = ' '.join(new_words)
        if debug:
            output += "(rd)"
        return output

In [15]:
## Word based
# class RandomDeletionAugmentor(BaseTextAugmentor):
#     def augment(self, text, n=1, debug=False):
#         words = text.split()
#         total_words = len(words)

#         # If there's only one word, or no words to delete, return the original text
#         if total_words == 1 or n == 0:
#             return text

#         # Ensure that the number of words to delete does not exceed the total number of words
#         n = min(int(n), total_words)  # Convert n to an integer

#         # Randomly select indices of words to delete
#         indices_to_delete = set(random.sample(range(total_words), n))

#         new_words = [word for i, word in enumerate(words) if i not in indices_to_delete]

#         # If all words are deleted, randomly select one to return
#         if len(new_words) == 0:
#             return words[random.randint(0, total_words - 1)]

#         output = ' '.join(new_words)
#         if debug:
#             output += "(rd)"
#         return output


# Random Swap

In [16]:
class RandomSwapAugmentor(BaseTextAugmentor):
    def swap_word(self, new_words):
        random_idx_1 = random.randint(0, len(new_words) - 1)
        random_idx_2 = random.randint(0, len(new_words) - 1)
        while random_idx_2 == random_idx_1:
            random_idx_2 = random.randint(0, len(new_words) - 1)
        new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
        return new_words

    def augment(self, text, n, debug=False):
        words = text.split()
        new_words = words.copy()
        for _ in range(n):
            new_words = self.swap_word(new_words)
        output =  ' '.join(new_words)
        if debug:
            output += "(rs)"
        return output

# Random Insertion

In [17]:
class RandomInsertionAugmentor(BaseTextAugmentor):
    def __init__(self, clean_text):
        super().__init__(clean_text)

    def add_word(self, new_words):
        synonyms = []
        counter = 0
        while len(synonyms) < 1:
            random_word = new_words[random.randint(0, len(new_words)-1)]
            synonyms = get_synonyms(random_word)
            counter += 1
            if counter >= 10:
                return
        random_synonym = synonyms[0]
        random_idx = random.randint(0, len(new_words)-1)
        new_words.insert(random_idx, random_synonym)

    def augment(self, text, n, debug=False):
        words = text.split()
        new_words = words.copy()
        for _ in range(n):
            self.add_word(new_words)
        output =  ' '.join(new_words)
        if debug:
            output +=  ' '.join(new_words) + "(ri)"
        return output

In [18]:
import random

# Assuming CleanText and other Augmentor classes are defined elsewhere

class TextAugmentationPipeline:
    def __init__(self, augmentors):
        self.augmentors = augmentors

    def augment(self, text, num_aug=9, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, p_paraphrase=0.1, p_bt=0.1, p_mlm=0.1, debug=False):
        text = self.augmentors[0].clean_text.remove_punctuations(text)  # Assuming remove_punctuations is a method in CleanText

        words = text.split()
        num_words = len(words)

        augmented_sentences = []
        num_new_per_technique = int(num_aug / len(self.augmentors)) + 1

        if alpha_sr > 0:
            n_sr = max(1, int(alpha_sr * num_words))
            for _ in range(num_new_per_technique):
                augmented_sentences.append(self.augmentors[0].augment(text, n_sr, debug))

        if p_rd > 0:
            n_rd = max(1, int(p_rd * num_words))
            for _ in range(num_new_per_technique):
                augmented_sentences.append(self.augmentors[1].augment(text, p_rd, debug)) # Probability based
                # augmented_sentences.append(self.augmentors[1].augment(text, n_rd, debug)) # word based

        if alpha_rs > 0:
            n_rs = max(1, int(alpha_rs * num_words))
            for _ in range(num_new_per_technique):
                augmented_sentences.append(self.augmentors[2].augment(text, n_rs, debug))

        if alpha_ri > 0:
            n_ri = max(1, int(alpha_ri * num_words))
            for _ in range(num_new_per_technique):
                augmented_sentences.append(self.augmentors[3].augment(text, n_ri, debug))


        if p_paraphrase > 0:
            for _ in range(num_new_per_technique):
                augmented_sentences.append(self.augmentors[4].augment(text, debug))

        if p_bt > 0:
            for _ in range(num_new_per_technique):
                augmented_sentences.append(self.augmentors[5].augment(text, debug))

        if p_mlm > 0:
            n_mlm = max(2, int(p_mlm * num_words))
            for _ in range(num_new_per_technique):
                augmented_sentences.append(self.augmentors[6].augment(text, n_mlm, debug))

        random.shuffle(augmented_sentences)

        if num_aug >= 1:
            augmented_sentences = augmented_sentences[:num_aug]
        else:
            keep_prob = num_aug / len(augmented_sentences)
            augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

        if debug:
            augmented_sentences.append(text + "(original)")

        return augmented_sentences



# Assuming CleanText and augmentor classes are defined/imported

# Usage
clean_text = CleanText()
stopwords = set(corpus.stopwords)

augmentors = [
    SynonymReplacementAugmentor(clean_text, stopwords),
    RandomDeletionAugmentor(clean_text),
    RandomSwapAugmentor(clean_text),
    RandomInsertionAugmentor(clean_text),
    BanglaParaphraseAugmentor(clean_text),
    BTAugmentor(clean_text),
    MLMAugmentor(clean_text)
]

pipeline = TextAugmentationPipeline(augmentors)


config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.15k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/623 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/528k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/140M [00:00<?, ?B/s]

In [19]:
text = "সঠিক তদন্ত করতে হবে। বিচারের আওতায় আনতে হবে যে এই কাজ টা করেছে।"
ct = CleanText()
text = ct.remove_punctuations(text)
augmented_sentences = pipeline.augment(text, debug=True)

Extracting: bangla_word2vec_gen4.zip:   0%|          | 0/8 [00:00<?, ?it/s]

In [20]:
for sentence in augmented_sentences:
    print(sentence)

সঠিক তদন্ত করতে হবে বিচারের আওতায় আনতে ভিত্তিতে হবে যে এই কাজ টা করেছেসঠিক তদন্ত করতে হবে বিচারের আওতায় আনতে ভিত্তিতে হবে যে এই কাজ টা করেছে(ri)
সঠিক যুদ্ধাপরাধ করতে হবে বিচারের আওতায় আনতে হবে যে এই কাজ টা করেছে(sr)
সঠিক তদন্ত অবশ্যই করতে হবে এবং যে ব্যক্তি এটা করেছে তাকে বিচারের আওতায় আনতে হবে।(bt)
দ্রুত তদন্ত করতে হবে বিচারের আওতায় আনতে হবে যে এই কাজ কে করেছে(mlm)
সঠিক তদন্ত করতে হবে এবং এই কাজটি বিচারের আওতায় আনতে হবে যে এটি করেছে।(pp)
সঠিক তদন্ত করতে হবে এবং এই কাজটি বিচারের আওতায় আনতে হবে যে এটি করেছে।(pp)
সঠিক তদন্ত করতে হবে বিচারের আওতায় আনতে হবে যে পুলিশ কাজ টা করেছে(mlm)
সঠিক তদন্ত করতে হবে বিচারের আওতায় সরাতে হবে যে এই কাজ টা করেছে(sr)
সঠিক তদন্ত অবশ্যই করতে হবে এবং যে ব্যক্তি এটা করেছে তাকে বিচারের আওতায় আনতে হবে।(bt)
সঠিক তদন্ত করতে হবে বিচারের আওতায় আনতে হবে যে এই কাজ টা করেছে(original)


# ScareBLEU

In [21]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 3.1 MB/s eta 0:00:00


In [22]:
import sacrebleu
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Example reference sentence in Bangla
reference = text  # This is a hypothetical example sentence in Bangla

# List of augmented candidate sentences in Bangla
candidates = augmented_sentences

# Compute BLEU score for each candidate
scarebleu_scores = [sacrebleu.sentence_bleu(candidate, [reference]).score for candidate in candidates]

# Compute Cosine Similarity
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform([reference] + candidates)
cosine_similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()

# Print the candidate sentences, their BLEU and Cosine Similarity scores
for i, (score, cos_sim) in enumerate(zip(scarebleu_scores, cosine_similarities)):
    print(f"Candidate {i + 1}: {candidates[i]} - ScareBLEU Score: {score}, Cosine Similarity: {cos_sim}")


Candidate 1: সঠিক তদন্ত করতে হবে বিচারের আওতায় আনতে ভিত্তিতে হবে যে এই কাজ টা করেছেসঠিক তদন্ত করতে হবে বিচারের আওতায় আনতে ভিত্তিতে হবে যে এই কাজ টা করেছে(ri) - ScareBLEU Score: 34.2104577056443, Cosine Similarity: 0.9286074029149645
Candidate 2: সঠিক যুদ্ধাপরাধ করতে হবে বিচারের আওতায় আনতে হবে যে এই কাজ টা করেছে(sr) - ScareBLEU Score: 66.68954865619207, Cosine Similarity: 0.6372843679648397
Candidate 3: সঠিক তদন্ত অবশ্যই করতে হবে এবং যে ব্যক্তি এটা করেছে তাকে বিচারের আওতায় আনতে হবে।(bt) - ScareBLEU Score: 12.512236921161914, Cosine Similarity: 0.475166361607639
Candidate 4: দ্রুত তদন্ত করতে হবে বিচারের আওতায় আনতে হবে যে এই কাজ কে করেছে(mlm) - ScareBLEU Score: 59.687741756345, Cosine Similarity: 0.7716835166235806
Candidate 5: সঠিক তদন্ত করতে হবে এবং এই কাজটি বিচারের আওতায় আনতে হবে যে এটি করেছে।(pp) - ScareBLEU Score: 36.821398145189974, Cosine Similarity: 0.6312234722364501
Candidate 6: সঠিক তদন্ত করতে হবে এবং এই কাজটি বিচারের আওতায় আনতে হবে যে এটি করেছে।(pp) - ScareBLEU Score: 3

# Sentence BanglaBert

In [23]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=c847b81d6763504e5b8e9a39c987664789eb17862bd5bba5ccd2ee6a1ced9f9f
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [24]:
from sentence_transformers import SentenceTransformer, util
import torch

# Function to calculate cosine similarity
def cosine_similarity(embedding1, embedding2):
    return torch.nn.functional.cosine_similarity(embedding1, embedding2, dim=0)

# Initialize the model
model_name = 'l3cube-pune/bengali-sentence-bert-nli'
model = SentenceTransformer(model_name)

# The reference sentence
reference_sentence = text
# List of sentences to compare with the reference
sentences = augmented_sentences

# Encode the sentences to get their embeddings
reference_embedding = model.encode(reference_sentence, convert_to_tensor=True)
sentence_embeddings = model.encode(sentences, convert_to_tensor=True)

# Calculate cosine similarities and pair with sentences
sentence_similarity_pairs = [(sentence, cosine_similarity(reference_embedding, embedding).item()) for sentence, embedding in zip(sentences, sentence_embeddings)]

# Sort the pairs based on similarity scores
sorted_pairs = sorted(sentence_similarity_pairs, key=lambda x: x[1], reverse=True)

# Print the sorted sentences and their scores
for sentence, score in sorted_pairs:
    print(f"Sentence: {sentence}\nSBERT Similarity: {score}\n")


.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.94k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/6.41M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/519 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

Sentence: সঠিক তদন্ত করতে হবে বিচারের আওতায় আনতে হবে যে এই কাজ টা করেছে(original)
SBERT Similarity: 0.9772871732711792

Sentence: সঠিক তদন্ত করতে হবে এবং এই কাজটি বিচারের আওতায় আনতে হবে যে এটি করেছে।(pp)
SBERT Similarity: 0.9579160809516907

Sentence: সঠিক তদন্ত করতে হবে এবং এই কাজটি বিচারের আওতায় আনতে হবে যে এটি করেছে।(pp)
SBERT Similarity: 0.9579160809516907

Sentence: সঠিক তদন্ত করতে হবে বিচারের আওতায় আনতে ভিত্তিতে হবে যে এই কাজ টা করেছেসঠিক তদন্ত করতে হবে বিচারের আওতায় আনতে ভিত্তিতে হবে যে এই কাজ টা করেছে(ri)
SBERT Similarity: 0.9544210433959961

Sentence: সঠিক তদন্ত অবশ্যই করতে হবে এবং যে ব্যক্তি এটা করেছে তাকে বিচারের আওতায় আনতে হবে।(bt)
SBERT Similarity: 0.909210741519928

Sentence: সঠিক তদন্ত অবশ্যই করতে হবে এবং যে ব্যক্তি এটা করেছে তাকে বিচারের আওতায় আনতে হবে।(bt)
SBERT Similarity: 0.909210741519928

Sentence: সঠিক তদন্ত করতে হবে বিচারের আওতায় সরাতে হবে যে এই কাজ টা করেছে(sr)
SBERT Similarity: 0.8977932333946228

Sentence: সঠিক যুদ্ধাপরাধ করতে হবে বিচারের আওতায় আনতে হ